# Natural Language Processing with Neural Networks

Using Krohn (2020)

In [ ]:
# import dependencies
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')
import nltk.stem as stemmer

import string

import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

from sklearn.manifold import TSNE

import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure
%matplotlib inline

In [ ]:
# get the dataset
from nltk.corpus import gutenberg

In [ ]:
# tokenize the corpus into a list of sentences
gberg_sent_tokens = sent_tokenize(gutenberg.raw())

In [ ]:
# tokenize further into sentences as list of lists
gberg_sents = gutenberg.sents()

In [ ]:
# convert all to lowercase
[w.lower() for w in gberg_sents[4]]

In [ ]:
# remove stop words and punctuation
stpwrds = stopwords.words('english') + list(string.punctuation)
[w.lower() for w in gberg_sents[4] if w.lower() not in stpwrds]

In [ ]:
# stemming words using the Porter algorithm built in to NLTK
[stemmer.stem(w.lower()) for w in gberg_sents[4]
if w.lower() not in stpwrds]

In [ ]:
#handling n-grams so that something like New York is counted as one word and not two 
phrases = Phrases(gberg_sents)
bigram = Phraser(phrases)

In [ ]:
# tokenizing a sentence by using split method
tokenized_sentence = 'John lives in New York City'.split()
print(tokenized_sentence)

Now that we've gone through some basic preprocessing requests, we can easily preprocess the entire corpus in one step rather than so many tedious steps

In [ ]:
lower_sents = []
for s in gberg_sents:
    lower_sents.append([w.lower() for w in s if w.lower() 
                       not in list(string.punctuation)])

In [ ]:
# too low of a threshold in the traditional way of scoring bigrams, so we need to cap what is considered a bigram
lower_bigram = Phraser(Phrases(lower_sents, min_count=32, threshold = 64))

In [ ]:
# Now we can create a clean corpus that includes bigrams 
clean_sents = []
for s in lower_sents:
    clean_sents.append(lower_bigram[s])

In [ ]:
clean_sents

Now that we have a clean corpus of data, we can embed words from the corpus into the word-vector space. word2vec uses predictive models, GloVe uses count models and is good across multiple NLP applications, fastText is good for subword level, so good for rare words and out of vocabulary words

In [ ]:
# we will use word2vec here
# sentences is a list of lists as a corpus
# size is the number of dimensions that the word vector space will result
# sg set to 1 to choose the skip-gram architecture or leave at 0 for CBOW (this is a small corpus, so using SG)
# window is good to be 10 for sg and 5 for CBOW, means the context words considered
# iter by default is 5, multiple iters is like multiple epochs
# min_count is the number of times a word must occur in order to fit into the word vector space
# workers is the number of processing cores committed to this task
model = Word2Vec(sentences=clean_sents,
                sg=1, window=10,
                min_count=10, workers=4)

In [ ]:
model.save('clean_gutenberg_model.w2v')

In [ ]:
# can load up prebuilt vectors instead too
model = gensim.models.Word2Vec.load('clean_gutenberg_model.w2v')

In [ ]:
# most similar method checks to make sure the word vectors are quality
model.wv.most_similar('father', topn=3)

In [ ]:
# check which word does not belong
model.wv.doesnt_match("mother father sister brother dog".split())

In [ ]:
# check familiarity score
model.wv.similarity('father', 'dog')

In [ ]:
# compute vfather - vman + vwoman to test the word vector analogies
model.wv.most_similar(positive=['father', 'woman'], negative =['man'])

In [ ]:
model.wv.most_similar(positive=['husband', 'woman'], negative =['man'])

t-distributed stochastic neighbor embedding (t-SNE) means we can compress vectors to two or three dimension

In [ ]:
# n_components is the number of dimensions that should be returned
# n_iter is the number of iterations over the input data, analogous to epochs associated with training neural network

tsne = TSNE(n_components=2, n_iter=1000)
X_2d = tsne.fit_transform(model.wv[model.wv.vocab])
coords_df = pd.DataFrame(X_2d, columns=['x','y'])
coords_df['token']=model.wv.vocab.keys()

In [ ]:
coords_df = pd.read_csv('clean_gutenberg_tsne.csv')
coords_df.head()
_ = coords_df.plot.scatter('x','y', figsize=(12,12), marker='.', s=10, alpha=0.2)

In [ ]:
# can also have interactive bokeh plot
output_notebook()
subset_df=coords_df.sample(n=5000)
p=figure(plot_width=800, plot_height=800)
_=p.text(x=subset_df.x, y=subset_df.y, text=subset_df.token)
show(p)